In [9]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.common.exceptions import NoSuchElementException
import pandas as pd
import time

In [10]:
with open('Ações.txt') as file:
    ações = file.readlines()

ações = [ação.strip() for ação in ações]
print(ações)

['ABEV3', 'ALUP11', 'B3SA3', 'BEEF3', 'DEXP3', 'EGIE3', 'MGLU3', 'MRFG3', 'PCAR3', 'PETR4', 'SAPR11', 'SEER3', 'TAEE11', 'TRPL4', 'USIM5', 'VALE3', 'VBBR3', 'PRIO3', 'WEGE3', 'WIZC3', 'TIMS3', 'VIVT3']


In [11]:
options = webdriver.ChromeOptions()
# options.add_argument('--headless')
# options.add_argument('--no-sandbox')
# options.add_argument('--disable-dev-shm-usage')

In [12]:
def _calcular_pontuacao(valor, faixas, pontos, peso, nome_indicador):
    """
    Calcula a pontuação de um indicador de acordo com as faixas e pontos.

    Args:
        valor: O valor do indicador (pode ser um único valor ou uma lista).
        faixas: Lista com os limites das faixas.
        pontos: Lista com as pontuações de cada faixa.
        peso: Peso do indicador.
        nome_indicador: Nome do indicador.

    Returns:
        int: Pontuação do indicador.
    """
    if isinstance(valor, list):
        # Calcula a pontuação para cada valor da lista
        pontuacoes = []
        for v in valor:
            if v < 0:
                pontuacao = -10  # Atribui -10 para valores negativos
            else:
                pontuacao = _calcular_pontuacao(v, faixas, pontos, peso, nome_indicador)
            print(f"{nome_indicador}: Valor: {v:.2f}, Pontuação: {pontuacao:.2f}")  # Imprime a pontuação de cada valor
            pontuacoes.append(pontuacao)
        # Retorna a média das pontuações
        return sum(pontuacoes) / len(pontuacoes)
    else:
        # Se valor não for uma lista, calcula a pontuação como antes
        for i in range(len(faixas)):
            if i == 0 and valor <= faixas[i]:
                return pontos[i] * peso
            elif valor > faixas[i] and (i+1 < len(faixas) and valor <= faixas[i+1]):
                return pontos[i+1] * peso
            elif valor > faixas[-1]:
                return pontos[-1] * peso
        return pontos[-1] * peso

def avaliar_acao(roe, margem_bruta, ebit, div_liq, lucro_liq, cagr, pl, p_vp, dy, payout, 
                 free_float, prejuizo, tag_along, estatal, fundacao, ibov, valor_de_mercado):
    """
    Calcula a pontuação de uma ação de acordo com a estratégia de valor e imprime os scores de cada critério.

    Args:
        roe (list): Lista com os valores de ROE dos últimos 5 anos.
        margem_bruta (float): Margem bruta da empresa.
        ebit (int): Lucro antes de juros e impostos.
        div_liq (int): Dívida líquida da empresa.
        lucro_liq (int): Lucro líquido da empresa.
        cagr (list): Lista com os valores de CAGR das receitas dos últimos 5 anos.
        pl (float): P/L (Preço/Lucro) da empresa.
        p_vp (float): P/VP (Preço/Valor Patrimonial) da empresa.
        dy (list): Lista com os valores de Dividend Yield dos últimos 5 anos.
        payout (list): Lista com os valores de Payout dos últimos 5 anos.
        free_float (float): Free Float da empresa.
        prejuizo (bool): True se a empresa teve prejuízo, False caso contrário.
        tag_along (float): Tag Along da empresa.
        estatal (str): "GOVERNO" se a empresa é estatal, "não" caso contrário.
        fundacao (int): Ano de fundação da empresa.
        ibov (float): Liquidez do mercado da empresa (ibov).
        valor_de_mercado (int): Valor de mercado da empresa.

    Returns:
        float: Pontuação da ação de acordo com a estratégia de valor.
    """

    # Rentabilidade
    roe_score = _calcular_pontuacao(roe, 
                                   [0.1, 0.15, 0.2, 0.25], 
                                   [1, 3, 5, 7, 10], 3, "ROE")
    print(f"ROE Score: {roe_score:.2f}")

    # Lucratividade
    margem_bruta_score = _calcular_pontuacao(margem_bruta, 
                                             [0.2, 0.3, 0.4, 0.5], 
                                             [1, 3, 5, 7, 10], 2, "Margem Bruta")
    print(f"Margem Bruta Score: {margem_bruta_score:.2f}")
    ebit_score = _calcular_pontuacao(ebit, 
                                      [0, 0.1, 0.2, 0.3], 
                                      [1, 3, 5, 7, 10], 2, "EBIT")
    print(f"EBIT Score: {ebit_score:.2f}")

    # Estrutura de Capital
    div_liq_lucro_liq_score = _calcular_pontuacao(div_liq/lucro_liq, 
                                                 [1, 0.8, 0.5, 0.2], 
                                                 [1, 3, 5, 7, 10], 2, "Dívida Líquida / Lucro Líquido")
    print(f"Dívida Líquida / Lucro Líquido Score: {div_liq_lucro_liq_score:.2f}")

    # Crescimento
    cagr_score = _calcular_pontuacao(cagr, 
                                     [0.05, 0.1, 0.15, 0.2], 
                                     [1, 3, 5, 7, 10], 2, "CAGR")
    print(f"CAGR Score: {cagr_score:.2f}")

    # Valuation
    pl_score = _calcular_pontuacao(pl, 
                                    [20, 15, 10, 5], 
                                    [1, 3, 5, 7, 10], 3, "P/L")
    print(f"P/L Score: {pl_score:.2f}")
    p_vp_score = _calcular_pontuacao(p_vp, 
                                     [5, 2, 1, 0.5], 
                                     [1, 3, 5, 7, 10], 1, "P/VP")
    print(f"P/VP Score: {p_vp_score:.2f}")

    # Dividendos
    dy_score = _calcular_pontuacao(dy, 
                                   [0.1, 0.15, 0.2, 0.25], 
                                   [1, 3, 5, 7, 10], 1, "Dividend Yield")
    print(f"Dividend Yield Score: {dy_score:.2f}")
    payout_score = _calcular_pontuacao(payout, 
                                       [0.2, 0.4, 0.6, 0.8], 
                                       [1, 3, 5, 7, 10], 1, "Payout")
    print(f"Payout Score: {payout_score:.2f}")

    # Liquidez
    free_float_score = _calcular_pontuacao(free_float, 
                                          [0.1, 0.2, 0.3, 0.4], 
                                          [1, 3, 5, 7, 10], 1, "Free Float")
    print(f"Free Float Score: {free_float_score:.2f}")

    # Risco
    prejuizo_score = 1 if not prejuizo else 10
    print(f"Prejuízo Score: {prejuizo_score:.2f}")
    tag_along_score = _calcular_pontuacao(tag_along, 
                                          [0.5, 0.75, 0.9, 1], 
                                          [1, 3, 5, 7, 10], 2, "Tag Along")
    print(f"Tag Along Score: {tag_along_score:.2f}")

    # Outros
    estatal_score = 10 if estatal == "não" else 1
    print(f"Estabilidade Score: {estatal_score:.2f}")
    fundacao_score = _calcular_pontuacao(fundacao, 
                                         [1, 5, 10], 
                                         [1, 3, 7, 10], 1, "Fundação")
    print(f"Fundação Score: {fundacao_score:.2f}")
    ibov_score = _calcular_pontuacao(ibov, 
                                    [0.01, 0.05, 0.1], 
                                    [1, 3, 7, 10], 1, "Liquidez do Mercado")
    print(f"Liquidez do Mercado Score: {ibov_score:.2f}")
    valor_de_mercado_score = _calcular_pontuacao(valor_de_mercado, 
                                                [1e9, 5e9, 10e9], 
                                                [1, 3, 7, 10], 1, "Tamanho do Mercado")
    print(f"Tamanho do Mercado Score: {valor_de_mercado_score:.2f}")

    # Pontuação Total
    pontuacao_total = (roe_score * 3 + margem_bruta_score * 2 + ebit_score * 2 + 
                       div_liq_lucro_liq_score * 2 + cagr_score * 2 + pl_score * 3 + 
                       p_vp_score * 1 + dy_score * 1 + payout_score * 1 + 
                       free_float_score * 1 + prejuizo_score * 3 + tag_along_score * 2 + 
                       estatal_score * 2 + fundacao_score * 1 + ibov_score * 1 + 
                       valor_de_mercado_score * 1)

    print(f"\nPontuação Total: {pontuacao_total:.2f}")

    return pontuacao_total / 29



In [13]:
def scroll_to_element(driver, scroll_value, max_scroll_value, scroll_step, xpath, extra_scroll=1000):
    
    while scroll_value <= max_scroll_value:
        try:
            # Rolar a página
            driver.execute_script(f'window.scroll(0,{scroll_value});')
            # Encontrar o elemento
            element = driver.find_element(By.XPATH, xpath)
            # Encontrou o elemento, adiciona o scroll extra
            driver.execute_script(f'window.scroll(0,{scroll_value + extra_scroll});')
            # Encontrou o elemento, retorna True
            return True

        except:
            # Aumenta o valor de scroll
            scroll_value += scroll_step

    # Não encontrou o elemento, retorna False
    return False

In [14]:

def aplicar_criterio(condicao, contador):
    
    if condicao:
        contador += 1
    else:
        contador -= 1
    return contador

In [15]:
def aplicar_2_criterio(condicao, condicao_2, contador):
    
    if condicao:
        contador += 1
    elif condicao_2:
        contador += 0
    else:
        contador -= 1
    return contador

In [16]:
driver = webdriver.Chrome(options=options)
#driver.maximize_window()

# Cria um DataFrame vazio para armazenar os resultados
df_resultados = pd.DataFrame(columns=['Ação', 'Nota'])

for ação in ações:
    contador = 0

    ##### fundamentus #####
    driver.get('https://www.fundamentus.com.br/detalhes.php?papel={}'.format(ação))

    div_liq = int(driver.find_element(By.XPATH, '/html/body/div[1]/div[2]/table[4]/tbody/tr[3]/td[4]/span').text.replace(".", "")) 
    lucro_liq = int(driver.find_element(By.XPATH, '/html/body/div[1]/div[2]/table[5]/tbody/tr[5]/td[2]/span').text.replace(".", "")) 
    p_vp = float(driver.find_element(By.XPATH, '/html/body/div[1]/div[2]/table[3]/tbody/tr[3]/td[4]/span').text.replace(",", "."))
    ebit = int(driver.find_element(By.XPATH, '/html/body/div[1]/div[2]/table[5]/tbody/tr[4]/td[2]/span').text.replace(".", ""))
    pl = float(driver.find_element(By.XPATH, '/html/body/div[1]/div[2]/table[3]/tbody/tr[2]/td[4]/span').text.replace(",", "."))
    variacao = [
        driver.find_element(By.XPATH, '/html/body/div[1]/div[2]/table[3]/tbody/tr[3]/td[2]/span/font').text,
        driver.find_element(By.XPATH, '/html/body/div[1]/div[2]/table[3]/tbody/tr[4]/td[2]/span/font').text,
        driver.find_element(By.XPATH, '/html/body/div[1]/div[2]/table[3]/tbody/tr[5]/td[2]/span/font').text
    ]  

    variacao = [(float(x[:-1].replace(',', '.'))) for x in variacao]

    margem_bruta = float(driver.find_element(By.XPATH, '/html/body/div[1]/div[2]/table[3]/tbody/tr[4]/td[6]/span').text.replace(",", ".").replace("%", ""))
    


    ##### investidor 10 #####
    driver.get('https://investidor10.com.br/acoes/{}/'.format(ação))

    # Define o valor inicial e o passo do loop
    scroll_value = 1000
    scroll_step = 500

    # Define o valor máximo de scroll (ajuste conforme necessário)
    max_scroll_value = 15000

    
    try:
        driver.execute_script('window.scroll(0,3300);')
        time.sleep(1)
        driver.find_element(By.XPATH, '//*[@id="indicators-history"]/button').click()
    except:
        driver.execute_script('window.scroll(0,2300);')
        time.sleep(1)
        driver.find_element(By.XPATH, '//*[@id="indicators-history"]/button').click()


    cagr = [
        driver.find_element(By.XPATH, '//*[@id="table-indicators-history"]/tbody/tr[30]/td[3]').text,
        driver.find_element(By.XPATH, '//*[@id="table-indicators-history"]/tbody/tr[30]/td[4]').text,
        driver.find_element(By.XPATH, '//*[@id="table-indicators-history"]/tbody/tr[30]/td[5]').text,
        driver.find_element(By.XPATH, '//*[@id="table-indicators-history"]/tbody/tr[30]/td[6]').text,
        driver.find_element(By.XPATH, '//*[@id="table-indicators-history"]/tbody/tr[30]/td[7]').text
    ]

    cagr =  list(map(lambda x: int(x.replace("-", "0").replace("%", "").split(',')[0]), cagr))

    dy = [
        driver.find_element(By.XPATH, '//*[@id="table-indicators-history"]/tbody/tr[5]/td[3]').text,
        driver.find_element(By.XPATH, '//*[@id="table-indicators-history"]/tbody/tr[5]/td[4]').text,
        driver.find_element(By.XPATH, '//*[@id="table-indicators-history"]/tbody/tr[5]/td[5]').text,
        driver.find_element(By.XPATH, '//*[@id="table-indicators-history"]/tbody/tr[5]/td[6]').text,
        driver.find_element(By.XPATH, '//*[@id="table-indicators-history"]/tbody/tr[5]/td[7]').text
    ]

    dy = [(float(x[:-1].replace(',', '.'))) for x in dy]

    payout = [
        driver.find_element(By.XPATH, '//*[@id="table-indicators-history"]/tbody/tr[6]/td[3]').text,
        driver.find_element(By.XPATH, '//*[@id="table-indicators-history"]/tbody/tr[6]/td[4]').text,
        driver.find_element(By.XPATH, '//*[@id="table-indicators-history"]/tbody/tr[6]/td[5]').text,
        driver.find_element(By.XPATH, '//*[@id="table-indicators-history"]/tbody/tr[6]/td[6]').text,
        driver.find_element(By.XPATH, '//*[@id="table-indicators-history"]/tbody/tr[6]/td[7]').text
    ]

    payout = [(float(x[:-1].replace(',', '.'))) for x in payout]

    # Rolar a página e clicar nos elementos
    if scroll_to_element(driver, scroll_value, max_scroll_value, scroll_step, '//*[@id="info_about"]/div[1]/div/ul/li[2]/span/span[1]/span/span[2]'):
        time.sleep(1)
        driver.find_element(By.XPATH, '//*[@id="info_about"]/div[1]/div/ul/li[2]/select/option[2]').click()

    #governaça 
    mercado = driver.find_element(By.XPATH, '//*[@id="table-indicators-company"]/div[10]/span[2]').text #Segmento de Listagem
    tag_along = float(driver.find_element(By.XPATH, '//*[@id="table-indicators-company"]/div[12]/span[2]').text.replace("-", "0.0").replace(",", ".").replace("%", ""))
    free_float = float(driver.find_element(By.XPATH, '//*[@id="table-indicators-company"]/div[11]/span[2]').text.replace(",", ".").replace("%", ""))
    
    #Tamanho blue chip
    valor_de_mercado = int(driver.find_element(By.XPATH, '//*[@id="table-indicators-company"]/div[1]/span[2]/div[2]').text.replace(".", "").split(' ')[1])
    

    # Rolar a página e clicar nos elementos
    if scroll_to_element(driver, scroll_value, max_scroll_value, scroll_step, '//*[@id="results_table"]/div[1]/div/ul/li[2]/span/span[1]/span/span[2]'):
        driver.find_element(By.XPATH, '//*[@id="results_table"]/div[1]/div/ul/li[2]/select/option[2]').click()

    lucro_liq5 = [
        driver.find_element(By.XPATH, '//*[@id="table-balance-results"]/tbody/tr[5]/td[3]/div[2]').text.replace("-", "0.0"),
        driver.find_element(By.XPATH, '//*[@id="table-balance-results"]/tbody/tr[5]/td[5]/div[2]').text.replace("-", "0.0"),
        driver.find_element(By.XPATH, '//*[@id="table-balance-results"]/tbody/tr[5]/td[8]/div[2]').text.replace("-", "0.0"),
        driver.find_element(By.XPATH, '//*[@id="table-balance-results"]/tbody/tr[5]/td[11]/div[2]').text.replace("-", "0.0"),
        driver.find_element(By.XPATH, '//*[@id="table-balance-results"]/tbody/tr[5]/td[14]/div[2]').text.replace("-", "0.0"),
    ]

    lucro_liq5 = list(map(lambda x: int(x.replace(".", "")), lucro_liq5))

    # Rolar a página e clicar nos elementos
    if scroll_to_element(driver, scroll_value, max_scroll_value, scroll_step, '//*[@id="assets-liabilities"]/div[1]/div/ul/li[2]/span/span[1]/span/span[2]'):
        driver.find_element(By.XPATH, '//*[@id="assets-liabilities"]/div[1]/div/ul/li[2]/select/option[2]').click()

    patri_liq5 = [
        driver.find_element(By.XPATH, '//*[@id="table-balance-sheet"]/tbody/tr[9]/td[2]/div[2]').text.replace("-", "0.0"),
        driver.find_element(By.XPATH, '//*[@id="table-balance-sheet"]/tbody/tr[9]/td[4]/div[2]').text.replace("-", "0.0"),
        driver.find_element(By.XPATH, '//*[@id="table-balance-sheet"]/tbody/tr[9]/td[7]/div[2]').text.replace("-", "0.0"),
        driver.find_element(By.XPATH, '//*[@id="table-balance-sheet"]/tbody/tr[9]/td[10]/div[2]').text.replace("-", "0.0"),
        driver.find_element(By.XPATH, '//*[@id="table-balance-sheet"]/tbody/tr[9]/td[13]/div[2]').text.replace("-", "0.0"),
    ]

    patri_liq5 = list(map(lambda x: int(x.replace(".", "")), patri_liq5))

    roe = [x / y for x, y in zip(lucro_liq5, patri_liq5)]
    
    fundacao = 2024 - int(driver.find_element(By.XPATH, '//*[@id="data_about"]/div[2]/div/div[1]/table/tbody/tr[5]/td[2]').text)

    prejuizo = driver.find_element(By.XPATH, '//*[@id="styled-checkbox-profitable"]').get_attribute('outerHTML')


    driver.get('https://fundamentei.com/br/{}/'.format(ação))
    
    estatal = "não"
    try:
        estatal = driver.find_element(By.XPATH, '//*[@id="__next"]/div[2]/div/div[1]/div[1]/div[2]/div[1]/div[2]/div/div[3]/div[1]/ul/li[1]/div[2]/span[2]').text

    except NoSuchElementException:
        pass # ou continue se quiser pular para a próxima ação

    try:
        ibov = float(driver.find_element(By.XPATH, '//*[@id="__next"]/div[2]/div/div[1]/div[4]/div[4]/div/div[2]/h1').text.replace("-", "0.0").replace(",", ".").replace("%", ""))

    except NoSuchElementException:
        ibov = 0 # ou continue se quiser pular para a próxima ação

    # Aplica os critérios para o contador
    print("")
    print(driver.title)
    #1. Rentabilidade e Desempenho:
    contador = aplicar_criterio(all(x > 0.05 for x in roe), contador)
    print(f"O ROE é {roe}")
    print(contador)

    contador = aplicar_criterio(margem_bruta > 40, contador)
    print(f"A margem_bruta é {margem_bruta}")
    print(contador)

    contador = aplicar_criterio("checked" in prejuizo, contador)
    print(f"A empresa nunca teve prejuizo? {"checked" in prejuizo}")
    print(contador)

    contador = aplicar_criterio(ebit > 0, contador)
    print(f"O ebit é {ebit}")
    print(contador)

    #2. Estrutura de Capital:
    contador = aplicar_criterio(div_liq < 0, contador)
    print(f"A div_liq é {div_liq}") 
    print(contador)
    
    contador = aplicar_criterio((abs(lucro_liq) - abs(div_liq)) > 0, contador) # Critério Dívida vs Lucro
    print(f"O lucro_liq vs div_liq é {abs(lucro_liq) - abs(div_liq)}")
    print(contador)

    #3. Crescimento:
    contador = aplicar_criterio(all(x > 5 for x in cagr), contador) 
    print(f"O cagr é {cagr}")
    print(contador)
    
    #4. Valuation:
    contador = aplicar_criterio(0 < pl < 20, contador) #Critério P/L (Preço/Lucro)
    print(f"O pl é {pl}")
    print(contador)
    
    contador = aplicar_2_criterio(0 < p_vp <= 2, 2 < p_vp <= 5, contador) #P/VP (Preço/Valor Patrimonial)
    print(f"O p_vp é {p_vp}")
    print(contador)

    #5. Governança Corporativa:
    contador = aplicar_2_criterio(tag_along >= 80, 50 <= tag_along < 80, contador)
    print(f"O tag_along é {tag_along}")
    print(contador)

    contador = aplicar_2_criterio(free_float >= 80, 40 <= free_float < 80, contador)
    print(f"O free_float é {free_float}")
    print(contador)

    contador = aplicar_criterio(fundacao > 30, contador)  # Critério Fundação
    print(f"A fundação é {fundacao}")
    print(contador)

    contador = aplicar_criterio(estatal != "GOVERNO", contador)  # Critério Estatal
    print(f"O estatal é {estatal}")
    print(contador)

    #6. Dividendos 
    contador = aplicar_2_criterio(all(x > 5 for x in dy), (sum(dy)/len(dy)) > 5, contador) # Critério DY
    print(f"O dy é {dy}")
    print(contador)

    contador = aplicar_2_criterio(all(x > 50 for x in payout), (sum(payout)/len(payout)) > 50, contador) # Critério DY
    print(f"O payout é {payout}")
    print(contador)

    #7. Mercado
    contador = aplicar_2_criterio(all(-15 < x <= 15 for x in variacao), all(-15 < x <= 50 for x in variacao), contador)  # Critério vies
    print(f"A variação é {variacao}")
    print(contador)

    contador = aplicar_criterio(valor_de_mercado > 100000000000 and ibov > 1, contador)
    print(f"O valor_de_mercado é {valor_de_mercado}")
    print(f"O ibov é {ibov}")
    print(contador)
    
    # Salva a nota no DataFrame
    df_resultados = pd.concat([df_resultados, pd.DataFrame({'Ação': [ação], 'Nota': [contador]})], ignore_index=True)

    # Imprime os valores das variáveis

    pontuacao = avaliar_acao(roe, margem_bruta, ebit, div_liq, lucro_liq, cagr, pl, p_vp, dy, payout, 
                        free_float, prejuizo, tag_along, estatal, fundacao, ibov, valor_de_mercado)

    print(f"A Nova pontuação da ação é: {pontuacao:.2f}")
    
    
# Fecha o navegador
driver.quit()

# Imprime os resultados no final
print("")
print(df_resultados)

df_resultado_ajust = df_resultados.copy()
df_resultado_ajust['Nota'] = df_resultado_ajust['Nota'].apply(lambda x: x if x >= 1 else (0.5 if x < 1 else x))

# Calcula a soma das notas ajustadas
soma = df_resultado_ajust['Nota'].sum()

# Cria o novo DataFrame com as colunas 'Ação' e 'Recomendação'
df_recomendacao = pd.DataFrame({'Ação': df_resultado_ajust['Ação'], 
                                'Recomendação': df_resultado_ajust['Nota'] / soma })

# Imprime os DataFrames
print(df_resultado_ajust)
print(df_recomendacao)


ABEV3 - Ambev | Fundamentei
O ROE é [0.18666974396847308, 0.17870746617575406, 0.1561884280940117, 0.15611091358319076, 0.19482316727041893]
1
A margem_bruta é 50.7
2
A empresa nunca teve prejuizo? True
3
O ebit é 16881500000
4
A div_liq é -10104300000
5
O lucro_liq vs div_liq é 4398300000
6
O cagr é [9, 10, 9, 4, 6]
5
O pl é 13.45
6
O p_vp é 2.25
6
O tag_along é 80.0
7
O free_float é 27.89
6
A fundação é 19
5
O estatal é não
6
O dy é [5.32, 5.25, 4.41, 2.64, 2.63]
5
O payout é [79.32, 83.06, 84.58, 57.27, 65.61]
6
A variação é [2.31, 3.69, -11.47]
7
O valor_de_mercado é 195032456000
O ibov é 2.64
8
ROE: Valor: 0.19, Pontuação: 15.00
ROE: Valor: 0.18, Pontuação: 15.00
ROE: Valor: 0.16, Pontuação: 15.00
ROE: Valor: 0.16, Pontuação: 15.00
ROE: Valor: 0.19, Pontuação: 15.00
ROE Score: 15.00
Margem Bruta Score: 20.00
EBIT Score: 20.00
Dívida Líquida / Lucro Líquido Score: 2.00
CAGR: Valor: 9.00, Pontuação: 20.00
CAGR: Valor: 10.00, Pontuação: 20.00
CAGR: Valor: 9.00, Pontuação: 20.00
CAGR

Função para nota considerando pesos para cada critério 

## Sistema de Avaliação de Ações - Estratégia de Valor (com Variáveis Adaptadas)

### Categorias e Indicadores:

**Rentabilidade:**

* **ROE (Return on Equity):** 
    * Cálculo: Média ponderada dos últimos 5 anos (utilizando a lista 'roe'), com maior peso para os anos mais recentes.
    * Critério de Pontuação:
        * 0-10%: 1 ponto
        * 10-15%: 3 pontos
        * 15-20%: 5 pontos
        * 20-25%: 7 pontos
        * Acima de 25%: 10 pontos
    * Peso: 3 (Alto)

**Lucratividade:**

* **Margem Bruta:** 
    * Cálculo: Utilizar o valor da variável 'margem_bruta'.
    * Critério de Pontuação:
        * 0-20%: 1 ponto
        * 20-30%: 3 pontos
        * 30-40%: 5 pontos
        * 40-50%: 7 pontos
        * Acima de 50%: 10 pontos
    * Peso: 2 (Médio)

* **EBIT (Lucro antes de juros e impostos):**
    * Cálculo: Utilizar o valor da variável 'ebit'.
    * Critério de Pontuação:
        * EBIT negativo: 1 ponto
        * 0 - 10% da receita: 3 pontos
        * 10 - 20% da receita: 5 pontos
        * 20 - 30% da receita: 7 pontos
        * Acima de 30% da receita: 10 pontos
    * Peso: 2 (Médio)

**Estrutura de Capital:**

* **Dívida Líquida / Lucro Líquido:**
    * Cálculo: 'div_liq' / 'lucro_liq'.
    * Critério de Pontuação:
        * Acima de 1: 1 ponto
        * 0.8-1: 3 pontos
        * 0.5-0.8: 5 pontos
        * 0.2-0.5: 7 pontos
        * Menos de 0.2: 10 pontos
    * Peso: 2 (Médio)

**Crescimento:**

* **CAGR Receitas (5 anos):**
    * Cálculo: Média ponderada dos últimos 5 anos (utilizando a lista 'cagr'), com maior peso para os anos mais recentes.
    * Critério de Pontuação:
        * 0-5%: 1 ponto
        * 5-10%: 3 pontos
        * 10-15%: 5 pontos
        * 15-20%: 7 pontos
        * Acima de 20%: 10 pontos
    * Peso: 2 (Médio)

**Valuation:**

* **P/L (Preço/Lucro):**
    * Cálculo: Utilizar o valor da variável 'pl'.
    * Critério de Pontuação:
        * Acima de 20: 1 ponto
        * 15-20: 3 pontos
        * 10-15: 5 pontos
        * 5-10: 7 pontos
        * Menos de 5: 10 pontos
    * Peso: 3 (Alto)

* **P/VP (Preço/Valor Patrimonial):**
    * Cálculo: Utilizar o valor da variável 'p_vp'.
    * Critério de Pontuação:
        * Acima de 5: 1 ponto
        * 2-5: 3 pontos
        * 1-2: 5 pontos
        * 0.5-1: 7 pontos
        * Menos de 0.5: 10 pontos
    * Peso: 1 (Baixo)

**Dividendos:**

* **Dividend Yield (DY):**
    * Cálculo: Média ponderada dos últimos 5 anos (utilizando a lista 'dy'), com maior peso para os anos mais recentes.
    * Critério de Pontuação:
        * 0-10%: 1 ponto
        * 10-15%: 3 pontos
        * 15-20%: 5 pontos
        * 20-25%: 7 pontos
        * Acima de 25%: 10 pontos
    * Peso: 1 (Baixo)

* **Payout (Taxa de Pagamento de Dividendos):**
    * Cálculo: Média ponderada dos últimos 5 anos (utilizando a lista 'payout'), com maior peso para os anos mais recentes.
    * Critério de Pontuação:
        * 0-20%: 1 ponto
        * 20-40%: 3 pontos
        * 40-60%: 5 pontos
        * 60-80%: 7 pontos
        * Acima de 80%: 10 pontos
    * Peso: 1 (Baixo)

**Liquidez:**

* **Free Float (Ações Disponíveis para Negociação):**
    * Cálculo: Utilizar o valor da variável 'free_float'.
    * Critério de Pontuação:
        * Menos de 10%: 1 ponto
        * 10-20%: 3 pontos
        * 20-30%: 5 pontos
        * 30-40%: 7 pontos
        * Acima de 40%: 10 pontos
    * Peso: 1 (Baixo)

**Risco:**

* **Prejuízo:** 
    * Cálculo: Utilizar o valor da variável 'prejuizo'.
    * Critério de Pontuação:
        * True (Prejuízo): 1 ponto
        * False (Lucro): 10 pontos
    * Peso: 3 (Alto)

* **Tag Along:**
    * Cálculo: Utilizar o valor da variável 'tag_along'.
    * Critério de Pontuação:
        * 0 - 50%: 1 ponto
        * 50 - 75%: 3 pontos
        * 75 - 90%: 5 pontos
        * 90 - 100%: 7 pontos
        * Acima de 100%: 10 pontos
    * Peso: 2 (Médio)

**Outros:**

* **Estabilidade:**
    * Cálculo: Utilizar o valor da variável 'estatal'.
    * Critério de Pontuação:
        * "GOVERNO": 1 ponto
        * "não": 10 pontos
    * Peso: 2 (Médio)

* **Fundamentação:**
    * Cálculo: Utilizar o valor da variável 'fundacao'.
    * Critério de Pontuação:
        * Acima de 10 anos: 10 pontos
        * 5 - 10 anos: 7 pontos
        * 1 - 5 anos: 3 pontos
        * Menos de 1 ano: 1 ponto
    * Peso: 1 (Baixo)

* **Liquidez do Mercado:**
    * Cálculo: Utilizar o valor da variável 'ibov'.
    * Critério de Pontuação:
        * Acima de 10%: 10 pontos
        * 5 - 10%: 7 pontos
        * 1 - 5%: 3 pontos
        * Menos de 1%: 1 ponto
    * Peso: 1 (Baixo)

* **Tamanho do Mercado:**
    * Cálculo: Utilizar o valor da variável 'valor_de_mercado'.
    * Critério de Pontuação:
        * Acima de R$ 10 bilhões: 10 pontos
        * R$ 5 bilhões - R$ 10 bilhões: 7 pontos
        * R$ 1 bilhão - R$ 5 bilhões: 3 pontos
        * Menos de R$ 1 bilhão: 1 ponto
    * Peso: 1 (Baixo)

### Cálculo da Pontuação:

1. **Calcule a pontuação para cada indicador:**
    * Utilize o critério de pontuação e o valor do indicador (considerando a média ponderada para indicadores com histórico).
2. **Multiplique a pontuação do indicador pelo seu peso.**
3. **Some as pontuações de todos os indicadores.**
4. **Divida a pontuação total pela soma dos pesos dos indicadores.**

**O resultado final será uma pontuação que representa a atratividade da ação de acordo com a estratégia de valor.**

**Observações:**

* Os pesos dos indicadores podem ser ajustados de acordo com a preferência do investidor.
* As pontuações podem ser utilizadas para comparar diferentes ações e identificar as mais atraentes para investimento.
* É importante lembrar que este sistema é apenas um guia para tomada de decisão e não garante o sucesso em investimentos.